In [219]:
from datetime import date, datetime, timedelta, time
import pandas as pd
import numpy as np
import holidays
import math
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

In [220]:
from datetime import date, datetime, timedelta, time
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
import holidays
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.metrics import roc_curve, confusion_matrix, accuracy_score, recall_score, precision_score, mean_absolute_error
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline

In [224]:
df = pd.read_csv("../taxi_data/taxi_data_final.csv", index_col=[0])

In [225]:
#df = df[["rides", "PULocationID", "year", "month", "hour", "wday", "holiday"]]

In [226]:
df['rides'] = df['rides'].apply(lambda x: np.log10(x+1))

In [227]:
df['dt'] = df[['year', 'month', 'day', 'hour']].apply(lambda s : datetime(*s),axis = 1)

In [228]:
df = df[df.dt >= datetime(2018,7,1)]

In [229]:
train = df[df.dt < datetime(2019,7,1)]
test = df[df.dt >= datetime(2019,7,1)]

In [230]:
train = train.drop("dt", axis=1)
test = test.drop("dt", axis=1)

In [231]:
df = df.drop("dt", axis=1)

In [232]:
# Creating list for numerical predictors/features
# Since 'Survived' is our target variable we will exclude this feature from this list of numerical predictors 
num_features = list(df.columns[df.dtypes!=object])
num_features.remove('rides')

In [233]:
# Creating list for categorical predictors/features 
cat_features = list(df.columns[df.dtypes==object])
cat_features

[]

In [234]:
X_train = train.drop('rides', axis=1)
X_test = test.drop('rides', axis=1)
y_train = train["rides"]
y_test = test["rides"]

In [235]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [236]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

X_train shape: (1031455, 7)
X_test shape: (520956, 7)
y_train shape: (1031455,)
y_test shape: (520956,)


In [237]:
# Pipline for numerical features
num_pipeline = Pipeline([
    ('imputer_num', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])
# Pipeline for categorical features 
cat_pipeline = Pipeline([
    ('imputer_cat', SimpleImputer(strategy='constant', fill_value='missing')),
    ('1hot', OneHotEncoder(handle_unknown='ignore'))
])
# Complete pipeline
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
])
pipe_xgb = Pipeline([
    ('preprocessor', preprocessor),
    ('xgb', XGBRegressor(colsample_bytree=0.7, 
                         gamma=0.1, 
                         learning_rate=0.3, 
                         max_depth=15,
                         xgb__min_child_weight=10))
])

In [238]:
# Making predictions on the training set using cross validation
y_train_predicted = cross_val_predict(pipe_xgb, X_train, y_train, cv=5)

/Users/johannespastorek/opt/anaconda3/envs/nf/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



[14:14:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/johannespastorek/opt/anaconda3/envs/nf/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



[14:16:35] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/johannespastorek/opt/anaconda3/envs/nf/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



[14:18:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/johannespastorek/opt/anaconda3/envs/nf/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



[14:20:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/johannespastorek/opt/anaconda3/envs/nf/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



[14:22:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [239]:
pred = [int(np.power(10,x) - 1) for x in y_train_predicted]
train = [int(np.power(10,x) - 1) for x in y_train]

In [240]:
mean_absolute_error(train, pred)

17.925630298946633

In [241]:
# Defining parameter space for grid-search. Since we want to access the classifier step in our pipeline 
param_xgb = {'xgb__learning_rate': [0.3],
             'xgb__max_depth': [15],
             'xgb__min_child_weight': [10],
             'xgb__gamma': [0.1],
             'xgb__colsample_bytree': [0.7]
            }
grid_xgb = GridSearchCV(pipe_xgb, param_grid=param_xgb, cv=3, scoring='neg_mean_absolute_error', 
                           verbose=5, n_jobs=-1)

In [242]:
grid_xgb.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  2.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  2.4min finished
/Users/johannespastorek/opt/anaconda3/envs/nf/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/johannespastorek/opt/anaconda3/envs/nf/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning:

Series.base is deprecated and will be removed in a future version



[14:28:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer_num',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('std_scaler',
                                                                                          StandardScaler())]),
                                                                         ['PULocationID',
                                                                          'year',
                                                                          'month',
                                                                          'day',
                                                                

In [243]:
# Show best parameters
print('Best score:\n{:.2f}'.format(grid_xgb.best_score_))
print("Best parameters:\n{}".format(grid_xgb.best_params_))

Best score:
-0.14
Best parameters:
{'xgb__colsample_bytree': 0.7, 'xgb__gamma': 0.1, 'xgb__learning_rate': 0.3, 'xgb__max_depth': 15, 'xgb__min_child_weight': 10}


In [244]:
# Save best model as best_model
best_model = grid_xgb.best_estimator_['xgb']

In [245]:
# Preparing the test set 
preprocessor.fit(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [246]:
# Calculating the accuracy, recall and precision for the test set with the optimized model
y_test_predicted = best_model.predict(X_test_preprocessed)

In [247]:
test_pred = [int(np.power(10,x) - 1) for x in y_test_predicted]
test = [int(np.power(10,x) - 1) for x in y_test]

In [248]:
mean_absolute_error(test, test_pred)

15.57475295418423

In [249]:
X_test["trips"] = test

In [250]:
X_test["predicted_trips"] = test_pred

In [251]:
X_test.head()

,PULocationID,year,month,day,hour,wday,holiday,trips,predicted_trips
0,2,2019,7,9,1,1,0,1,0
1,2,2019,7,10,18,2,0,1,0
2,2,2019,7,10,21,2,0,1,2
3,2,2019,7,11,10,3,0,1,1
4,2,2019,7,13,15,5,0,1,1


In [252]:
X_test['dt'] = X_test[['year', 'month', 'day', 'hour']].apply(lambda s : datetime(*s),axis = 1)

In [256]:
X_test.to_csv("../taxi_data/predictions.csv")